In [1]:
import time

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import errno

from common.camera import *
from common.model import *
from common.loss import *
from common.utils import *
from common.visualization import *
from common.generators import ChunkedGenerator, UnchunkedGenerator
from common.utils import deterministic_random

from scripts.data_preprocessing_human36 import *
from scripts.data_preprocessing_cmu_mocap import *
from scripts.build_model import *
from scripts.train import *
from scripts.eval import *

import plotly
plotly.offline.init_notebook_mode(connected=True)

## Load and Preprocess Data

In [2]:
data_root_path = "../wild_data_cmu/output_human/" 

In [3]:
dataset, keypoints = load_and_preprocess_cmu_mocap(data_root_path)

 load 3d data 
 processing 3d data 
 load 2d keypoints 
 processing 2d keypoints 


## Generate Subsets

In [4]:
subjects_test = ['01']

In [5]:
cameras_valid, poses_valid, poses_valid_2d = fetch(dataset = dataset, 
                                                   keypoints = keypoints, 
                                                   subjects = subjects_test, 
                                                   stride = 1)

## Load Model

In [6]:
resume = True
filter_widths = [3,3,3,3,3]
model_pos_train, model_pos, model_traj, model_traj_train = build_models(17, 2, 17, filter_widths)

# chk_filename = "../checkpoint/pretrained_h36m_cpn.bin"
chk_filename = "checkpoints/cmu_epoch_100.bin"
print('Loading checkpoint', chk_filename)
checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
print('This model was trained for {} epochs'.format(checkpoint['epoch']))
model_pos_train.load_state_dict(checkpoint['model_pos'])
model_pos.load_state_dict(checkpoint['model_pos'])
model_traj_train.load_state_dict(checkpoint['model_traj'])
model_traj.load_state_dict(checkpoint['model_traj'])

Loading checkpoint checkpoints/cmu_epoch_100.bin
This model was trained for 100 epochs


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Test Data Generator

In [7]:
receptive_field = model_pos.receptive_field()
print('INFO: Receptive field: {} frames'.format(receptive_field))
pad = (receptive_field - 1) // 2 # Padding on each side

INFO: Receptive field: 243 frames


In [8]:
test_generator = UnchunkedGenerator([dataset.cameras()["05"][0]["intrinsics"]], 
                                    [dataset["05"]["01"]["positions_3d"][0]], 
                                    [keypoints["05"]["01"][0]], 
                                    pad=pad, 
                                    causal_shift=0, 
                                    augment=False)

## Inference

In [9]:
with torch.no_grad():
    model_pos.eval()
    N = 0
    for cam, batch, batch_2d in test_generator.next_epoch():
        inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
        inputs_3d = torch.from_numpy(batch.astype('float32'))
        cam = torch.from_numpy(cam.astype('float32'))
        # Positional model
        predicted_3d_pos = model_pos(inputs_2d)  
        pred_traj = model_pos(inputs_2d).numpy()
        prediction = predicted_3d_pos.numpy()
        gt_cam = inputs_3d.numpy()
        gt_2d = inputs_2d.numpy()

In [10]:
prediction = prediction[0]
gt_cam = gt_cam[0]
gt_2d = gt_2d[0]

## Post Processing

In [11]:
trajectory = gt_cam[:, :1]
gt_cam[:, 1:] += trajectory
prediction += pred_traj[0]

In [12]:
# camera to world
gt_3d_world = camera2world_cv(gt_cam,
                              dataset.cameras()["05"][0]["rvec"], 
                              dataset.cameras()["05"][0]["tvec"])

In [13]:
np.sum(np.power(gt_3d_world - dataset["05"]["01"]["positions"], 2))

7.953271035710611e-10

In [14]:
# camera to world
pred_3d_world = camera2world_cv(prediction,
                              dataset.cameras()["01"][0]["rvec"], 
                              dataset.cameras()["01"][0]["tvec"])

In [15]:
human36m_kpts_name = ['Pelvis', 'RHip', 'RKnee', 'RAnkle','LHip','LKnee',
                      'LAnkle','Spine1','Neck', 'Head','Site','LShoulder',
                      'LElbow','LWrist','RShoulder', 'RElbow','RWrist']

In [16]:
obj_pts = gt_3d_world.reshape(-1, 3)
xmin = obj_pts[:,0].min(); xmax = obj_pts[:,0].max()
ymin = obj_pts[:,1].min(); ymax = obj_pts[:,1].max()
zmin = obj_pts[:,2].min(); zmax = obj_pts[:,2].max()

frames, layout = generate_frames_layout(gt_3d_world[::50], 
                                        dataset.skeleton(), 
                                        human36m_kpts_name, 
                                        x_range = [xmin, xmax], 
                                        y_range = [ymin, ymax], 
                                        z_range = [zmin, zmax])

fig = dict(data=get_line_3d_segs(gt_3d_world, 0, dataset.skeleton(), human36m_kpts_name), 
           layout=layout, frames=frames)
plotly.offline.iplot(fig)

In [20]:
pred_3d_world.shape

(598, 17, 3)

In [21]:
obj_pts = pred_3d_world.reshape(-1, 3)
xmin = obj_pts[:,0].min(); xmax = obj_pts[:,0].max()
ymin = obj_pts[:,1].min(); ymax = obj_pts[:,1].max()
zmin = obj_pts[:,2].min(); zmax = obj_pts[:,2].max()

frames, layout = generate_frames_layout(pred_3d_world[::20], 
                                        dataset.skeleton(), 
                                        human36m_kpts_name, 
                                        x_range = [xmin, xmax], 
                                        y_range = [ymin, ymax], 
                                        z_range = [zmin, zmax])

fig = dict(data=get_line_3d_segs(pred_3d_world, 0, dataset.skeleton(), human36m_kpts_name), 
           layout=layout, frames=frames)
plotly.offline.iplot(fig)

In [19]:
# projection 2d
frames, layout = generate_frames_layout_2d(keypoints["05"]["01"][0][::50], dataset.skeleton(), human36m_kpts_name, x_range=[-1, 1], y_range=[1, -1])
fig = dict(data=get_line_2d_segs(keypoints["05"]["01"][0], 0, dataset.skeleton(), human36m_kpts_name), layout=layout, frames=frames)
plotly.offline.iplot(fig)